In [2]:
from utils import *

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [3]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [4]:
MODEL_NAMES = ['bert-base-uncased'] # ['bert-base-uncased', 'xlnet-base-cased']

In [6]:
import numpy as np

In [ ]:
np.load()

In [5]:
npy_load("./assets/AG_NEWS/topic/INV/text.npy")

ValueError: cannot reshape array of size 15479 into shape (120000,)

In [14]:
INV_text = npy_load("./assets/AG_NEWS/topic/INV/text.npy")
INV_label = npy_load("./assets/AG_NEWS/topic/INV/label.npy")

SIB_text = npy_load("./assets/AG_NEWS/topic/SIB/text.npy")
SIB_label = npy_load("./assets/AG_NEWS/topic/SIB/label.npy")

ValueError: cannot reshape array of size 15479 into shape (120000,)

In [ ]:
for MODEL_NAME in MODEL_NAMES:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    def tokenize(batch):
        return tokenizer(batch['text'], padding=True, truncation=True)

    train_dataset, test_dataset = load_dataset('ag_news', split=['train', 'test'])
    train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
    test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
    train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

    training_args = TrainingArguments(
        output_dir='./results/' + MODEL_NAME,
        overwrite_output_dir=True,
        max_steps=100000,
        save_steps=5000,
        save_total_limit=1,
        per_device_train_batch_size=3,
        per_device_eval_batch_size=3,
        warmup_steps=5000,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=1000
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=test_dataset
    )

    trainer.train()
    trainer.evaluate()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at